# 1. Lendo bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import os

# Regressão Linear
from sklearn import datasets, linear_model
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression


import seaborn as sns
%matplotlib inline
import seaborn as sns
from sklearn import metrics


# Plots
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display

# 2. Ler o DataSet

In [ ]:
filename = 'dataset.xlsx'
if not filename in os.listdir():
    print(f'Não encontrei o arquivo {filename}')

In [ ]:
dados = pd.read_excel(filename)
print(f"Temos as colunas {dados.columns}")

# 3. Tratamento da coluna 'Category'
A coluna category na prática é uma hierarquia de categorias que podem se misturar. Ex. A subcategoria celulares pode estar tanto dentro de comunicação quanto de tecnologia. Nesta etapa vamos separar a categoria em categoria, subcategoria e subsubcategoria.

In [ ]:
# dados['parentcategory', 'subcategory', 'subsubcategory'] = dados['category'].str.split(' > ')
dados['parentcategory'] = dados['category'].map(lambda category: category.split(' > ')[0])
dados['subcategory'] = dados['category'].map(lambda category: category.split(' > ')[1])
dados['subsubcategory'] = dados['category'].map(lambda category: category.split(' > ')[2] if 2 < len(category.split(' > ')) else "None")

dados.head()

# 4. Análise Exploratória
O primeiro passo após ter o DataSet carregado é olhar os dados e tentar descobrir possíveis correlações. Para isso vamos plotar alguns gráficos:

**Primeiramente buscamos a utilizar os dados da funcao Describe para assim plotar os graficos!**

In [ ]:

print(dados.describe(include = 'all'))

In [ ]:
# Todos os gráficos possíveis
sns.pairplot(dados,height = 5)


**Gráficos:** Esses gráficos representam nossa análise exploratória, abaixo refizemos os gráficos que consideramos os mais importantes

In [ ]:

plt.figure(figsize = (8,6))
plt.scatter(dados.rating,dados["price (£)"], c='red', alpha=0.4)
plt.title('Rating X Preço')
plt.xlabel('Rating')
plt.ylabel('Preço em euros')
plt.show()

plt.figure(figsize = (8,6))
plt.scatter(dados.number_of_reviews, dados.rating, c='red', alpha=0.4)
plt.title('Quantidade de Reviews X Rating')
plt.xlabel('Numero de Reviews')
plt.ylabel('Rating')

plt.show()

plt.figure(figsize = (8,6))
plt.scatter(dados.number_of_reviews, dados["price (£)"], c='red', alpha=0.4)
plt.title('Quantidade de Reviews X Preço')
plt.xlabel('Numero de Reviews')
plt.ylabel('Preço em Euros')

plt.figure(figsize = (8,6))
plt.hist(dados.rating, edgecolor = 'white', density = True, color = "red")
plt.title('Avaliações')
plt.xlabel('Nivel da avaliação')

plt.show()




## Observando os gráficos
Para a análise dos produtos, nós vamos usar três principais características: preço, avaliação e número de avaliações; sendo review o target e as demais variáveis features.

A partir dos gráficos acima formulamos a seguinte hipótese: **Quanto maior a avaliação, maior o preço e quanto maior número de reviews, menor o preço do produto**.
Lógica por trás da decisão:
1. Pessoas vão entrar para reclamar se o produto for ruim, mas não necessariamente vão elogiá-lo se for bom;
2. Produtos com boas avaliações tendem a ter melhor qualidade, o que reflete em seu preço

# Tentativa #1
Neste passo vamos fazer uma regressão linear multivariada utilizando MMQ, levando em consideração a o preço em função da avaliação e número de reviews.

In [ ]:
def regress(Y,X):
    '''
    Y: coluna do DataFrame utilizada como variável resposta (TARGET)
    X: coluna(s) do DataFrame utilizadas como variável(is) explicativas (FEATURES)
    '''
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    
    return results

In [ ]:
# Target é o preço
y = dados[['price (£)']]

# Vamos usar o preço e número de reviews como features
x = dados[['number_of_reviews', 'rating']]

# Agora vamos fazer MMQ (OLS) com a função regress
resultados = regress(y, x)
resultados.summary()

# Tentativa #2

In [ ]:
X_cp = sm.add_constant(x)
resultados.predict(X_cp)


In [ ]:
plt.figure(figsize = (8,6))
plt.scatter(dados.rating,resultados.predict(), c='blue', alpha=0.4)
plt.title('Previsão do preço X Rating')
plt.grid(True)
plt.xlabel('Rating')
plt.ylabel('Preço em Euros')

plt.figure(figsize = (8,6))
plt.grid(True)
plt.scatter(dados.number_of_reviews,resultados.predict(), c='red', alpha=0.4)
plt.xlabel('Number of Reviews')
plt.ylabel('Preço em Libras')

plt.show()

# Tentativa #3
Queremos aumentar a exatidão do nosso modelo; para isso vamos tentar adicionar mais features para aumentar esse número. O problema é que o MMQ não funciona com variáveis categóricas, então vamos usar

In [ ]:
# Target é o preço
y_2 = dados[['price (£)']]

# Vamos usar o preço e número de reviews como features
x_2 = dados[['number_of_reviews', 'rating', 'parentcategory', 'subcategory', 'subsubcategory']]

# Podemos separar a análise por categoria?

# Agora vamos fazer MMQ (OLS) com a função regress
resultados_2 = regress(y_2, x_2)
resultados_2.summary()

# Tentativa #4